In [2]:
!pip install nltk

In [4]:
import pandas as pd
import requests
import re
import nltk
import warnings
import csv
from io import StringIO

import sys
sys.path.append('/mnt/c/Users/cleon/Documents/CAL_CAL/ironhack/Final_project/Redefining_Cancer_treatment/src')
import datafunctions as dataf

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

from nltk.corpus import stopwords

warnings.filterwarnings("ignore");
nltk.download('stopwords');

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cleonortiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cleonortiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Preliminary exploration

In [63]:
trainvariants = pd.read_csv('../data/training_variants')
testvariants = pd.read_csv('../data/test_variants')
trainvariants.sample(5)

,ID,Gene,Variation,Class
2839,2839,BRCA2,D2723H,1
720,720,ERBB2,S310F,7
972,972,ESR1,V422del,7
2436,2436,BRCA1,M1689T,1
1338,1338,KMT2A,Truncating Mutations,1


- *training_variants* and *test_variants* are a comma separated file containing the description of the genetic mutations used for training.
- Fields are:
    - ID: the id of the row used to link the mutation to the clinical evidence
    - Gene: the gene where this genetic mutation is located
    - Variation: the aminoacid change for this mutations
    -Class: 1-9 the class this genetic mutation has been classified on. Classes are as follow:
    
        1. Likely loss-of-function
        2. Likely Gain-of-function
        3. Neutral
        4. Loss-of-function
        5. Likely Neutral
        6. Inconclusive
        7. Gain-of-function
        8. Likely Switch-of-function
        9. Switch-of-function

In [64]:
trainvariants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         3321 non-null   int64 
 1   Gene       3321 non-null   object
 2   Variation  3321 non-null   object
 3   Class      3321 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 103.9+ KB


In [65]:
traintext = pd.read_csv('../data/training_text',sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)
testtext = pd.read_csv('../data/test_text',sep="\|\|",engine="python",names=["ID","TEXT"],skiprows=1)

In [66]:
traintext.head()

,ID,TEXT
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [67]:
traintext.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      3321 non-null   int64 
 1   TEXT    3316 non-null   object
dtypes: int64(1), object(1)
memory usage: 52.0+ KB


In [68]:
traintext.head()

,ID,TEXT
0,0,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,Abstract Background Non-small cell lung canc...
2,2,Abstract Background Non-small cell lung canc...
3,3,Recent evidence has demonstrated that acquired...
4,4,Oncogenic mutations in the monomeric Casitas B...


In [69]:
trainvariants.Gene.unique()

array(['FAM58A', 'CBL', 'SHOC2', 'TERT', 'DICER1', 'PTPRT', 'RHEB',
       'SHQ1', 'CCND2', 'RAD50', 'CCND3', 'RIT1', 'CCNE1', 'RYBP',
       'TGFBR1', 'TGFBR2', 'MSH6', 'KMT2D', 'LATS1', 'PBRM1', 'SF3B1',
       'LATS2', 'EGFR', 'NKX2-1', 'EIF1AX', 'ARID2', 'BRD4', 'HIST1H1C',
       'ERRFI1', 'CHEK2', 'PAK1', 'TMPRSS2', 'H3F3A', 'ELF3', 'ROS1',
       'ASXL2', 'CDH1', 'EPCAM', 'EP300', 'EPAS1', 'TP53', 'TP53BP1',
       'SMAD2', 'SMAD3', 'SMAD4', 'CDK4', 'AURKB', 'CDK6', 'FBXW7',
       'CDK8', 'CDKN1A', 'CDKN1B', 'CDKN2A', 'CDKN2B', 'CDKN2C', 'ASXL1',
       'ERBB2', 'ERBB3', 'ERBB4', 'ERCC2', 'BRIP1', 'ERCC3', 'ERCC4',
       'ABL1', 'CEBPA', 'ERG', 'HLA-A', 'HLA-B', 'PDGFRA', 'PDGFRB',
       'RBM10', 'KDM5C', 'ESR1', 'ETV1', 'ETV6', 'TSC1', 'IKZF1', 'TSC2',
       'EWSR1', 'ACVR1', 'FOXA1', 'MDM2', 'ARID1A', 'EZH2', 'MDM4',
       'KLF4', 'RXRA', 'MAP3K1', 'AXIN1', 'BAP1', 'MEN1', 'FANCA',
       'FANCC', 'MET', 'U2AF1', 'KMT2C', 'FAT1', 'ARID1B', 'PIK3CA',
       'PIK3CB', 'PIM1

In [70]:
uvariants = trainvariants.Variation.unique().size
nvariants = trainvariants.Variation.size

print(f'Number of different variants: {uvariants}\nNumber of columns: {nvariants}')

Number of different variants: 2996
Number of columns: 3321


In [71]:
trainvariants = dataf.typeeffect(trainvariants)
testvariants = dataf.typeeffect(testvariants)

In [72]:
trainvariants.sample(5)

,ID,Gene,Variation,Class,Type,Effect
2914,2914,NFE2L2,E79Q,7,Substitution,missense
1361,1361,AKT1,D325A,7,Substitution,missense
1039,1039,TSC2,R462C,1,Substitution,missense
2456,2456,BRCA1,P1749R,4,Substitution,missense
2154,2154,PTEN,D107Y,4,Substitution,missense


In [73]:
testvariants.sample(5)

,ID,Gene,Variation,Type,Effect
4286,4286,CBS,P422L,Substitution,missense
4469,4469,EVC2,R870W,Substitution,missense
669,669,SIAE,T277M,Substitution,missense
4336,4336,SHH,G290D,Substitution,missense
283,283,RPS6KA3,F268S,Substitution,missense


In [74]:
trainvariants = trainvariants[['ID', 'Gene', 'Variation', 'Type', 'Effect', 'Class']]
testvariants = testvariants[['ID', 'Gene', 'Variation', 'Type', 'Effect']]

trainvariants.set_index('ID',inplace=True)
testvariants.set_index('ID',inplace=True)
trainvariants.head()

In [75]:
trainvariants.to_csv('../trainvariantswithtypeandeffect.csv')
testvariants.to_csv('../testvariantswithtypeandeffect.csv')

In [76]:
result = pd.merge(trainvariants, traintext, on='ID', how='left')
testresult = pd.merge(testvariants, testtext, on='ID', how='left')
result.head()

,ID,Gene,Variation,Type,Effect,Class,TEXT
0,0,FAM58A,Truncating Mutations,unknown,nonsense,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,Substitution,nonsense,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,Substitution,missense,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,Substitution,missense,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,Substitution,missense,4,Oncogenic mutations in the monomeric Casitas B...


In [77]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3321 entries, 0 to 3320
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         3321 non-null   int64 
 1   Gene       3321 non-null   object
 2   Variation  3321 non-null   object
 3   Type       3321 non-null   object
 4   Effect     3321 non-null   object
 5   Class      3321 non-null   int64 
 6   TEXT       3316 non-null   object
dtypes: int64(2), object(5)
memory usage: 207.6+ KB


In [78]:
result.loc[result['TEXT'].isnull(),'TEXT'] = result['Gene'] + ' ' + result['Variation']
testresult.loc[testresult['TEXT'].isnull(),'TEXT'] = testresult['Gene'] + ' ' + testresult['Variation']

In [79]:
result.head()

,ID,Gene,Variation,Type,Effect,Class,TEXT
0,0,FAM58A,Truncating Mutations,unknown,nonsense,1,Cyclin-dependent kinases (CDKs) regulate a var...
1,1,CBL,W802*,Substitution,nonsense,2,Abstract Background Non-small cell lung canc...
2,2,CBL,Q249E,Substitution,missense,2,Abstract Background Non-small cell lung canc...
3,3,CBL,N454D,Substitution,missense,3,Recent evidence has demonstrated that acquired...
4,4,CBL,L399V,Substitution,missense,4,Oncogenic mutations in the monomeric Casitas B...


In [80]:
testresult.head()

,ID,Gene,Variation,Type,Effect,TEXT
0,0,ACSL4,R570S,Substitution,missense,2. This mutation resulted in a myeloproliferat...
1,1,NAGLU,P521L,Substitution,missense,Abstract The Large Tumor Suppressor 1 (LATS1)...
2,2,PAH,L333F,Substitution,missense,Vascular endothelial growth factor receptor (V...
3,3,ING1,A148D,Substitution,missense,Inflammatory myofibroblastic tumor (IMT) is a ...
4,4,TMEM216,G77A,Substitution,missense,Abstract Retinoblastoma is a pediatric retina...


## 2. Data enrichment

### 2.1. Cancerous, GeneType and Chromosome based on an external dataset

In [81]:
oncogene = pd.read_csv('../data/ongene_human2.csv',sep="\t")

In [22]:
def cancerous(train, oncogene):
    
    train['Cancerous'] = 'Unknown'
    train['GeneType'] = 'Unknown'
    train['Chromosome'] = 0
    cuenta = 0
    for itrain,rtrain in train.iterrows():
        gene = rtrain['Gene']
        
        for ionco,ronco in oncogene.iterrows():
            if ronco['OncogeneName'] == gene:
                train['Cancerous'][itrain] = 'Oncogene'
                train['GeneType'][itrain] = ronco['GeneType']
                train['Chromosome'][itrain] = ronco['Cytoband'].split('q')[0]
                cuenta +=1
    print(f'{cuenta} genes had been clasified')
    
    return train

In [23]:
oncogene.head()

,OncogeneID,OncogeneName,Alias,Links,Cytoband,FullName,GeneType
0,10221,TRIB1,C8FW|GIG-2|GIG2|SKIP1|TRB-1|TRB1,MIM:609461|HGNC:HGNC:16891|Ensembl:ENSG0000017...,8q24.13,tribbles pseudokinase 1,protein-coding
1,7295,TXN,TRDX|TRX|TRX1,MIM:187700|HGNC:HGNC:12435|Ensembl:ENSG0000013...,9q31,thioredoxin,protein-coding
2,867,CBL,C-CBL|CBL2|FRA11B|NSLL|RNF55,MIM:165360|HGNC:HGNC:1541|Ensembl:ENSG00000110...,11q23.3,"Cbl proto-oncogene, E3 ubiquitin protein ligase",protein-coding
3,673,BRAF,B-RAF1|BRAF1|NS7|RAFB1,MIM:164757|HGNC:HGNC:1097|Ensembl:ENSG00000157...,7q34,"B-Raf proto-oncogene, serine/threonine kinase",protein-coding
4,5979,RET,CDHF12|CDHR16|HSCR1|MEN2A|MEN2B|MTC1|PTC|RET-E...,MIM:164761|HGNC:HGNC:9967|Ensembl:ENSG00000165...,10q11.2,ret proto-oncogene,protein-coding


In [24]:
# nuevo =cancerous(result,oncogene)

In [25]:
# nuevo.to_csv('nuevo.csv')

In [26]:
# testnuevo = cancerous(testresult,oncogene)

In [27]:
# testnuevo.to_csv('testnuevo.csv')

### 2.2. Chromosome, Tumour_type, and Role using web scrapping with Selenium

In [96]:
sanger = pd.read_csv('../data/Census_allMon Dec 13 12_39_44 2021.tsv', sep = '\t')

In [97]:
sanger.head()

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"29974,A1CF,ACF,ACF64,ACF65,APOBEC1CF,ASP,ENSG0..."
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,12.10,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"10006,ABI-1,ABI1,E3B1,ENSG00000136754.17,Q8IZP..."
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"25,ABL,ABL1,ENSG00000097007.17,JTK7,P00519,c-A..."
3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,25.20,yes,NaN,AML,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"27,ABL2,ABLL,ARG,ENSG00000143322.19,P42684"
4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,37.30,yes,NaN,lipoma,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"57007,ACKR3,CMKOR1,CXCR7,ENSG00000144476.5,GPR..."


In [5]:
sanger.shape

(729, 20)

In [98]:
sanger.to_csv('../data/sanger.csv')

## 3. Merging data

In [82]:
data = pd.read_csv('nuevo.csv')

In [83]:
testdata = pd.read_csv('testnuevo.csv')

In [84]:
data.Chromosome = data.Chromosome.apply(lambda x: x.split('p')[0])

In [85]:
testdata.Chromosome = testdata.Chromosome.apply(lambda x: x.split('p')[0])

In [86]:
data.Chromosome.unique()

array(['0', '11', '1', '19', '7', '14', '21', '6', '16', '2', '12', '13',
       '17', '9', '4', '5', '22', '3', '8', '10', 'X', '15', '20', '18'],
      dtype=object)

In [87]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3321 non-null   int64 
 1   ID          3321 non-null   int64 
 2   Gene        3321 non-null   object
 3   Variation   3321 non-null   object
 4   Type        3321 non-null   object
 5   Effect      3321 non-null   object
 6   Class       3321 non-null   int64 
 7   TEXT        3321 non-null   object
 8   Cancerous   3321 non-null   object
 9   GeneType    3321 non-null   object
 10  Chromosome  3321 non-null   object
dtypes: int64(3), object(8)
memory usage: 285.5+ KB


In [88]:
testdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5668 entries, 0 to 5667
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5668 non-null   int64 
 1   ID          5668 non-null   int64 
 2   Gene        5668 non-null   object
 3   Variation   5668 non-null   object
 4   Type        5668 non-null   object
 5   Effect      5668 non-null   object
 6   TEXT        5668 non-null   object
 7   Cancerous   5668 non-null   object
 8   GeneType    5668 non-null   object
 9   Chromosome  5668 non-null   object
dtypes: int64(2), object(8)
memory usage: 442.9+ KB


In [110]:
sanger = pd.read_csv('../data/sanger.csv')

In [111]:
sanger.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729 entries, 0 to 728
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              729 non-null    int64  
 1   Gene Symbol             729 non-null    object 
 2   Name                    729 non-null    object 
 3   Entrez GeneId           725 non-null    float64
 4   Genome Location         729 non-null    object 
 5   Tier                    729 non-null    int64  
 6   Hallmark                318 non-null    object 
 7   Chr Band                729 non-null    float64
 8   Somatic                 686 non-null    object 
 9   Germline                108 non-null    object 
 10  Tumour Types(Somatic)   686 non-null    object 
 11  Tumour Types(Germline)  105 non-null    object 
 12  Cancer Syndrome         87 non-null     object 
 13  Tissue Type             729 non-null    object 
 14  Molecular Genetics      596 non-null    ob

In [112]:
sanger =sanger.rename({'Gene Symbol': 'Gene'},axis=1)

In [113]:
sanger

,Unnamed: 0,Gene,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,...,Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,11.23,yes,NaN,...,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"29974,A1CF,ACF,ACF64,ACF65,APOBEC1CF,ASP,ENSG0..."
1,1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,12.10,yes,NaN,...,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"10006,ABI-1,ABI1,E3B1,ENSG00000136754.17,Q8IZP..."
2,2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,34.12,yes,NaN,...,NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"25,ABL,ABL1,ENSG00000097007.17,JTK7,P00519,c-A..."
3,3,ABL2,"c-abl oncogene 2, non-receptor tyrosine kinase",27.0,1:179099327-179229601,1,NaN,25.20,yes,NaN,...,NaN,NaN,L,Dom,"oncogene, fusion",T,ETV6,NaN,NaN,"27,ABL2,ABLL,ARG,ENSG00000143322.19,P42684"
4,4,ACKR3,atypical chemokine receptor 3,57007.0,2:236569641-236582358,1,Yes,37.30,yes,NaN,...,NaN,NaN,M,Dom,"oncogene, fusion",T,HMGA2,NaN,NaN,"57007,ACKR3,CMKOR1,CXCR7,ENSG00000144476.5,GPR..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,724,ZNF429,zinc finger protein 429,353088.0,19:21505564-21538078,2,NaN,12.00,yes,NaN,...,NaN,NaN,O,NaN,NaN,Mis,NaN,NaN,NaN,"353088,ENSG00000197013.9,Q86V71,ZNF429"
725,725,ZNF479,zinc finger protein 479,90827.0,7:57119614-57139864,2,NaN,11.20,yes,NaN,...,NaN,NaN,E,NaN,NaN,Mis,NaN,NaN,NaN,"90827,ENSG00000185177.12,KR19,Q96JC4,ZNF479"
726,726,ZNF521,zinc finger protein 521,25925.0,18:25061926-25352152,1,NaN,11.20,yes,NaN,...,NaN,NaN,L,Dom,"oncogene, fusion",T,PAX5,NaN,NaN,"25925,EHZF,ENSG00000198795.10,Evi3,Q96K83,ZNF521"
727,727,ZNRF3,zinc and ring finger 3,84133.0,22:28883592-29057487,2,NaN,12.10,yes,NaN,...,NaN,NaN,E,NaN,TSG,"N, F, Mis",NaN,NaN,NaN,"84133,BK747E2.3,ENSG00000183579.15,FLJ22057,KI..."


In [132]:
mergedata= data.merge(sanger, how='left', on='Gene')
mergetestdata = testdata.merge(sanger, how='left', on = 'Gene')

In [133]:
mergedata.head(5)

,Unnamed: 0_x,ID,Gene,Variation,Type,Effect,Class,TEXT,Cancerous,GeneType,...,Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms
0,0,0,FAM58A,Truncating Mutations,unknown,nonsense,1,Cyclin-dependent kinases (CDKs) regulate a var...,Unknown,Unknown,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,CBL,W802*,Substitution,nonsense,2,Abstract Background Non-small cell lung canc...,Oncogene,protein-coding,...,NaN,NaN,L,Dom/Rec,"oncogene, TSG, fusion","T, Mis, S, O",KMT2A,NaN,NaN,"867,CBL,CBL2,ENSG00000110395.6,P22681,RNF55,c-Cbl"
2,2,2,CBL,Q249E,Substitution,missense,2,Abstract Background Non-small cell lung canc...,Oncogene,protein-coding,...,NaN,NaN,L,Dom/Rec,"oncogene, TSG, fusion","T, Mis, S, O",KMT2A,NaN,NaN,"867,CBL,CBL2,ENSG00000110395.6,P22681,RNF55,c-Cbl"
3,3,3,CBL,N454D,Substitution,missense,3,Recent evidence has demonstrated that acquired...,Oncogene,protein-coding,...,NaN,NaN,L,Dom/Rec,"oncogene, TSG, fusion","T, Mis, S, O",KMT2A,NaN,NaN,"867,CBL,CBL2,ENSG00000110395.6,P22681,RNF55,c-Cbl"
4,4,4,CBL,L399V,Substitution,missense,4,Oncogenic mutations in the monomeric Casitas B...,Oncogene,protein-coding,...,NaN,NaN,L,Dom/Rec,"oncogene, TSG, fusion","T, Mis, S, O",KMT2A,NaN,NaN,"867,CBL,CBL2,ENSG00000110395.6,P22681,RNF55,c-Cbl"


In [134]:
mergedata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3321 entries, 0 to 3320
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0_x            3321 non-null   int64  
 1   ID                      3321 non-null   int64  
 2   Gene                    3321 non-null   object 
 3   Variation               3321 non-null   object 
 4   Type                    3321 non-null   object 
 5   Effect                  3321 non-null   object 
 6   Class                   3321 non-null   int64  
 7   TEXT                    3321 non-null   object 
 8   Cancerous               3321 non-null   object 
 9   GeneType                3321 non-null   object 
 10  Chromosome              3321 non-null   object 
 11  Unnamed: 0_y            3188 non-null   float64
 12  Name                    3188 non-null   object 
 13  Entrez GeneId           3188 non-null   float64
 14  Genome Location         3188 non-null   

In [135]:
mergedata = mergedata[['ID', 'Gene','Variation', 'Type', 'Effect', 'Class', 'TEXT', 'Cancerous', 'GeneType', 'Genome Location', 'Role in Cancer', 'Tumour Types(Somatic)' ]]

In [136]:
mergetestdata = mergetestdata[['ID', 'Gene','Variation', 'Type', 'Effect', 'TEXT', 'Cancerous', 'GeneType', 'Genome Location', 'Role in Cancer', 'Tumour Types(Somatic)' ]]

In [137]:
mergedata.rename(columns={'Genome Location': 'Chromosome', 'Role in Cancer': 'Role','Tumour Types(Somatic)':'TumourType', 'Type':'VariationType' },inplace=True)
mergetestdata.rename(columns={'Genome Location': 'Chromosome', 'Role in Cancer': 'Role','Tumour Types(Somatic)':'TumourType', 'Type':'VariationType' },inplace=True)

In [138]:
mergedata.head()

,ID,Gene,Variation,VariationType,Effect,Class,TEXT,Cancerous,GeneType,Chromosome,Role,TumourType
0,0,FAM58A,Truncating Mutations,unknown,nonsense,1,Cyclin-dependent kinases (CDKs) regulate a var...,Unknown,Unknown,NaN,NaN,NaN
1,1,CBL,W802*,Substitution,nonsense,2,Abstract Background Non-small cell lung canc...,Oncogene,protein-coding,11:119206276-119308149,"oncogene, TSG, fusion","AML, JMML, MDS"
2,2,CBL,Q249E,Substitution,missense,2,Abstract Background Non-small cell lung canc...,Oncogene,protein-coding,11:119206276-119308149,"oncogene, TSG, fusion","AML, JMML, MDS"
3,3,CBL,N454D,Substitution,missense,3,Recent evidence has demonstrated that acquired...,Oncogene,protein-coding,11:119206276-119308149,"oncogene, TSG, fusion","AML, JMML, MDS"
4,4,CBL,L399V,Substitution,missense,4,Oncogenic mutations in the monomeric Casitas B...,Oncogene,protein-coding,11:119206276-119308149,"oncogene, TSG, fusion","AML, JMML, MDS"


## 4. Cleaning

In [144]:
mergedata.Chromosome = mergedata.Chromosome.fillna('unknown:')
mergetestdata.Chromosome = mergetestdata.Chromosome.fillna('unknown:')

mergedata.Chromosome = mergedata.Chromosome.apply(lambda x: x.split(':')[0])
mergetestdata.Chromosome = mergetestdata.Chromosome.apply(lambda x: x.split(':')[0])

mergedata['Role'] = mergedata['Role'].fillna(mergedata['Cancerous'])
mergetestdata['Role'] = mergetestdata['Role'].fillna(mergetestdata['Cancerous'])

mergedata['TumourType'] = mergedata['TumourType'].fillna('Unknown')
mergetestdata['TumourType'] = mergetestdata['TumourType'].fillna('Unknown')

mergedata.drop(columns=['Cancerous'], inplace=True)
mergetestdata.drop(columns=['Cancerous'], inplace=True)

In [145]:
mergedata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3321 entries, 0 to 3320
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             3321 non-null   int64 
 1   Gene           3321 non-null   object
 2   Variation      3321 non-null   object
 3   VariationType  3321 non-null   object
 4   Effect         3321 non-null   object
 5   Class          3321 non-null   int64 
 6   TEXT           3321 non-null   object
 7   GeneType       3321 non-null   object
 8   Chromosome     3321 non-null   object
 9   Role           3321 non-null   object
 10  TumourType     3321 non-null   object
dtypes: int64(2), object(9)
memory usage: 311.3+ KB


In [146]:
mergetestdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5668 entries, 0 to 5667
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             5668 non-null   int64 
 1   Gene           5668 non-null   object
 2   Variation      5668 non-null   object
 3   VariationType  5668 non-null   object
 4   Effect         5668 non-null   object
 5   TEXT           5668 non-null   object
 6   GeneType       5668 non-null   object
 7   Chromosome     5668 non-null   object
 8   Role           5668 non-null   object
 9   TumourType     5668 non-null   object
dtypes: int64(1), object(9)
memory usage: 487.1+ KB


In [147]:
mergedata.head()

,ID,Gene,Variation,VariationType,Effect,Class,TEXT,GeneType,Chromosome,Role,TumourType
0,0,FAM58A,Truncating Mutations,unknown,nonsense,1,Cyclin-dependent kinases (CDKs) regulate a var...,Unknown,unknown,Unknown,Unknown
1,1,CBL,W802*,Substitution,nonsense,2,Abstract Background Non-small cell lung canc...,protein-coding,11,"oncogene, TSG, fusion","AML, JMML, MDS"
2,2,CBL,Q249E,Substitution,missense,2,Abstract Background Non-small cell lung canc...,protein-coding,11,"oncogene, TSG, fusion","AML, JMML, MDS"
3,3,CBL,N454D,Substitution,missense,3,Recent evidence has demonstrated that acquired...,protein-coding,11,"oncogene, TSG, fusion","AML, JMML, MDS"
4,4,CBL,L399V,Substitution,missense,4,Oncogenic mutations in the monomeric Casitas B...,protein-coding,11,"oncogene, TSG, fusion","AML, JMML, MDS"


In [8]:
for index, row in mergedata.iterrows():
    if type(row['TEXT']) is str:
        dataf.remove_stopwords(row['TEXT'], index,mergedata)
    else:
        print("There is no text description for id:", index)

In [9]:
for index, row in mergetestdata.iterrows():
    if type(row['TEXT']) is str:
        dataf.remove_stopwords(row['TEXT'], index, mergetestdata)
    else:
        print("There is no text description for id:", index)

In [11]:
mergedata.to_csv('../data/trainclean.csv')
mergetestdata.to_csv('../data/testclean.csv')

In [ ]:
result.Gene = result.Gene.str.replace('\s+', '_')
result.Variation = result.Variation.str.replace('\s+', '_')

In [13]:
mergetestdata

,Unnamed: 0,ID,Gene,Variation,VariationType,Effect,TEXT,GeneType,Chromosome,Role,TumourType
0,0,0,ACSL4,R570S,Substitution,missense,2 mutation resulted myeloproliferative phenoty...,Unknown,unknown,Unknown,Unknown
1,1,1,NAGLU,P521L,Substitution,missense,abstract large tumor suppressor 1 lats1 serine...,Unknown,unknown,Unknown,Unknown
2,2,2,PAH,L333F,Substitution,missense,vascular endothelial growth factor receptor ve...,Unknown,unknown,Unknown,Unknown
3,3,3,ING1,A148D,Substitution,missense,inflammatory myofibroblastic tumor imt neoplas...,Unknown,unknown,Unknown,Unknown
4,4,4,TMEM216,G77A,Substitution,missense,abstract retinoblastoma pediatric retinal tumo...,Unknown,unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...
5663,5663,5663,SLC46A1,R113S,Substitution,missense,realization late 1970s ras harboured transform...,Unknown,unknown,Unknown,Unknown
5664,5664,5664,FOXC1,L130F,Substitution,missense,hemizygous deletions common molecular abnormal...,Unknown,unknown,Unknown,Unknown
5665,5665,5665,GSS,R267W,Substitution,missense,r267w smartpool investigate 533 experiments 5q...,Unknown,unknown,Unknown,Unknown
5666,5666,5666,CTSK,G79E,Substitution,missense,abstract blood samples 125 unrelated families ...,Unknown,unknown,Unknown,Unknown
